<img src="../images/ilmudatapy-logo.png" width="350" align="center">
<br>

<center><h1>Prediksi Customer Churn dengan Decision Tree</h1></center>
<hr>

__Halo, Learners!__ Di notebook ini, kita akan mempraktekkan pemodelan <i>machine learning</i> dengan algoritma berbasis pohon __Decision Tree__ untuk kasus klasifikasi. Kita juga akan mencoba memvisualisasikan pohon keputusan tersebut dengan menginstal <i>package</i> <code>pydotplus</code> dan <code>python-graphviz</code>.

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 25px">
    <ul>
        <li>
            Decision Tree
        </li>
        <li>
            Dataset
        </li>
        <li>
            Analisis dan visualisasi data
        </li>
        <li>
            Preprocessing
            <ul>
                <li>Menangani missing values</li>
                <li>Encoding</li>
                <li>Normalisasi</li>
                <li>Train test split</li>
            </ul>
        </li>
        <li>
            Modeling
            <ul>
                <li>Klasifikasi dengan Decision Tree</li>
                <li>Evaluasi</li>
                <li>Tree visualization</li>
            </ul>
        </li>
    </ul>
</div>

<hr>
<div class="alert alert-success" style="margin-top: 20px">
    <strong>Catatan:</strong> Untuk menjalankan kode program Python di Jupyter Notebook, klik pada <i>cell</i> yang ingin di-<i>run</i> lalu tekan <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>

<div class="alert alert-danger" style="margin-top: 20px">
    <strong>Warning!:</strong> Jika ada kode program yang <i>error</i> atau output yang dihasilkan tidak sesuai, silahkan <b>Restart & Run All</b> kernel pada bagian menu <b>Kernel</b> di menu bar Jupyter Notebook, atau <b>Restart & Clear Output</b> kernel kemudian jalankan satu per satu <i>cell</i> secara berurutan dari atas ke bawah.
</div>
<hr>

## Decision Tree

__Decision Tree__ adalah salah satu algoritma <i>machine learning</i> yang memiliki struktur seperti pohon yang dapat digunakan untuk kasus <i>supervised learning</i> baik klasifikasi maupun regresi, namun lebih sering digunakan untuk kasus klasifikasi. 

Dalam <i>decision tree</i> terdapat dua jenis node, yaitu __decision node__ dan __leaf node__. <i>Decision node</i> mewakili fitur dari dataset dan digunakan untuk membuat keputusan. <i>leaf node</i> adalah output dari keputusan tersebut dan tidak berisi cabang lebih lanjut.

![alt text](../images/decision-tree.png)

Ada dua cara untuk menentukan fitur terbaik yang akan dijadikan <i>root node</i> dan juga <i>sub-nodes</i>, yaitu __Information Gain__ (__Entropy__) dan __Gini Index__. 

* Information gain : Pengukuran perubahan entropy setelah segmentasi kumpulan data berdasarkan atribut.
* Gini index : Ukuran <i>impurity</i> atau <i>purity</i> yang digunakan ketika membuat <i>decision tree</i> dalam algoritma CART (Classification and Regression Tree).

Dalam implementasi di Python, kita dapat memberikan argumen <code>criterion = 'entropy'</code> atau <code>criterion = 'gini'</code>.

<hr>

## Dataset

Dataset yang digunakan adalah dataset <a href="https://www.kaggle.com/blastchar/telco-customer-churn">Telco Customer Churn</a> yang bertujuan untuk memprediksi apakah pelanggan berhenti menggunakan produk atau tidak.

Dataset ini terdiri dari sejumlah kolom yang berisi informasi tentang:

* Layanan yang digunakan pelanggan, yaitu kolom <code>PhoneService</code>, <code>MultipeLines</code>, <code>InternetService</code>, <code>OnlineSecurity</code>, <code>OnlineBackup</code>, <code>DeviceProtection</code>, <code>TechSupport</code>, <code>StreamingTV</code>, <code>StreamingMovies</code>.
* Informasi akun pelanggan, yaitu kolom <code>tenure</code>, <code>Contract</code>, <code>PaperlessBilling</code>, <code>PaymentMethod</code>, <code>MonthlyCharges</code>, <code>TotalCharges</code>.
* Informasi demografi pelanggan, yaitu kolom <code>gender</code>, <code>SeniorCitizen</code>, <code>Partner</code>, <code>Dependents</code>.

Target kolomnya yaitu kolom <code>Churn</code> yang berisi tentang data pelanggan yang berhenti menggunakan produk dalam sebulan terakhir.

Pertama mari kita <i>import library</i> terlebih dahulu.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Selanjutnya kita <i>load</i> dataset ke dalam dataframe Pandas.

In [ ]:
# Load dataset 

df = pd.read_csv('../datasets/Telco-Customer-Churn.csv')

pd.set_option('display.max_columns', 50)      # menampilkan seluruh kolom
df.head()

<hr>

## Analisis dan visualisasi data

Mari kita cek info dataframe <code>df</code>.

In [ ]:
# Cek info

df.info()

Selanjutnya kita dapat menampilkan visualisasi jumlah kelas/kategori pada kolom <code>Churn</code> dengan <code>countplot()</code>.

In [ ]:
# Memvisualisasikan jumlah data pada kolom 'Churn'

sns.countplot(df['Churn'], palette='Set1')

Sekarang kita visualisasikan jumlah data pada kolom <code>Churn</code> berdasarkan <code>gender</code>.

In [ ]:
# Visualisasi kolom 'Churn' berdasarkan 'gender'

sns.countplot(x='Churn', hue='gender', data=df, palette='Set1')

Kita dapat mengecek jumlah pelanggan yang melakukan kontrak perbulan, satu tahun, atau dua tahun dengan <code>value_counts()</code>.

In [ ]:
# Menampilkan jumlah data untuk tiap kategori di kolom 'Contract'

df['Contract'].value_counts()

Kita juga menampilkan visualisasi data <code>PaymentMethod</code> berdasarkan <code>gender</code>.

In [ ]:
# Menampilkan data 'PaymentMethod' berdasarkan 'gender'

plt.figure(figsize=(10,6))
sns.countplot(x='PaymentMethod', hue='gender', data=df, palette='Set2')

Selanjutnya kita dapat menganalisis langsung dua plot atau lebih dengan menggunakan subplot.

In [ ]:
# Menampilkan countplot

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(12, 6)) 

# Menambahkan subplot dengan indexing
ax0 = fig.add_subplot(ax[0]) 
ax1 = fig.add_subplot(ax[1])    

sns.countplot(x='Contract', hue='gender', data=df, palette='Set3', ax=ax0)
sns.countplot(x='InternetService', hue='gender', data=df, palette='Set3', ax=ax1)

plt.subplots_adjust(wspace=0.3)
plt.show()

In [ ]:
# Menampilkan countplot

fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(14, 12)) 

# Menambahkan subplot dengan indexing
ax0 = fig.add_subplot(ax[0,0]) 
ax1 = fig.add_subplot(ax[0,1])  
ax2 = fig.add_subplot(ax[0,2])  
ax3 = fig.add_subplot(ax[1,0]) 
ax4 = fig.add_subplot(ax[1,1])  
ax5 = fig.add_subplot(ax[1,2]) 
ax6 = fig.add_subplot(ax[2,0]) 
ax7 = fig.add_subplot(ax[2,1]) 
ax8 = fig.add_subplot(ax[2,2]) 

sns.countplot(x='MultipleLines', hue='gender', data=df, palette='Set3', ax=ax0)
sns.countplot(x='PhoneService', hue='gender', data=df, palette='Set3', ax=ax1)
sns.countplot(x='OnlineSecurity', hue='gender', data=df, palette='Set3', ax=ax2)
sns.countplot(x='OnlineBackup', hue='gender', data=df, palette='Set3', ax=ax3)
sns.countplot(x='DeviceProtection', hue='gender', data=df, palette='Set3', ax=ax4)
sns.countplot(x='TechSupport', hue='gender', data=df, palette='Set3', ax=ax5)
sns.countplot(x='StreamingTV', hue='gender', data=df, palette='Set3', ax=ax6)
sns.countplot(x='StreamingMovies', hue='gender', data=df, palette='Set3', ax=ax7)
sns.countplot(x='PaperlessBilling', hue='gender', data=df, palette='Set3', ax=ax8)

plt.subplots_adjust(wspace=0.3, hspace=0.4)
plt.show()

Untuk data numerik, kita dapat melihat histogramnya. Mari kita cek dahulu tipe datanya.

In [ ]:
# cek tipe data kolom numerik

df[['tenure', 'MonthlyCharges', 'TotalCharges']].dtypes

Karena kolom <code>TotalCharges</code> masih bertipe object, kita harus mengubah dulu tipe datanya. Disini kita menggunakan <code>to_numeric()</code> dari Pandas.

In [ ]:
# mengubah tipe data kolom TotalCharges menjadi numerik

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')     

Lalu kita tampilkan histogramnya.

In [ ]:
# Menampilkan histogram

fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(16, 5)) 

# Menambahkan subplot dengan indexing
ax0 = fig.add_subplot(ax[0]) 
ax1 = fig.add_subplot(ax[1])  
ax2 = fig.add_subplot(ax[2])   

# Subplot ax[0]: tenure
df.hist(column='tenure', bins=50, ax=ax0)

# Subplot ax[1]: MonthlyCharges
df.hist(column='MonthlyCharges', bins=50, ax=ax1)

# Subplot ax[2]: TotalCharges
df.hist(column='TotalCharges', bins=50, ax=ax2)

plt.subplots_adjust(wspace=0.2)
plt.show()

<hr>

## Data Preparation / Preprocessing

Dalam tahap <i>preprocessing</i> ini, kita akan terlebih dulu mendefinisikan data fitur dan target dari dataframe ini.

In [ ]:
# Mendefinisikan data fitur dan target

df_features = df.drop(['customerID', 'Churn'], axis=1)
df_target = df['Churn']

Mari kita cek <code>df_features</code>.

In [ ]:
# Menampilkan 5 data teratas

df_features.head()

### Menangani missing values

Selanjutnya kita harus cek apakah ada <i>missing values</i> dalam dataframe ini.

In [ ]:
# Mengecek missing values

df_features.isnull().sum()

Ternyata ada 11 <i>missing values</i> pada kolom <code>TotalCharges</code>, sehingga kita harus menanganinya terlebih dahulu. Disini kita akan mengisi nilai yang hilang tersebut dengan nilai rata-rata kolomnya.

In [ ]:
# Menangani missing values

df_features['TotalCharges'].fillna(df_features['TotalCharges'].mean(), inplace=True)

### Encoding

Kita akan melakukan <i>encoding</i> data terhadap data non-numerik menggunakan <code>.cat.codes</code>. Namun, sebelumnya kita harus mengubah dulu tipe data kolom non-numerik dari <i>object</i> menjadi <i>category</i>.

In [ ]:
cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
        'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
        'PaperlessBilling', 'PaymentMethod']

# mengubah tipe data kolom bertipe object menjadi category
df_features[cols] = df_features[cols].astype('category')

df_features.info()

Selanjutnya kita aplikasikan proses <i>encoding</i> dengan <code>.cat.codes</code>.

In [ ]:
# Data encoding

for col in cols:
    df_features[col] = df_features[col].cat.codes

df_features.head()

### Normalisasi

Kita juga harus menormalisasi dataframe tersebut agar <i>range</i> nilai antar kolomnya tidak terlalu jauh.

In [ ]:
from sklearn.preprocessing import StandardScaler

# normalisasi data / scaling data
scale = StandardScaler().fit_transform(df_features)

# mendefinisikan nama kolom
cols = list(df_features.columns)

# membuat dataframe untuk menampilkan hasil scaling pada dataframe
df_features_scale = pd.DataFrame(scale, columns=cols)
df_features_scale.head()

### Train test split

Selanjutnya kita melakukan <i>train test split</i>.

In [ ]:
from sklearn.model_selection import train_test_split

# Train test split
X_train, X_test, y_train, y_test = train_test_split(df_features_scale, df_target, test_size=0.2, random_state=3)

print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

<hr>

## Modeling 

### Klasifikasi dengan Decision Tree

Membuat model klasifikasi dengan <i>decision tree</i> dapat menggunakan <code>DecisionTreeClassifier</code>.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_dtree = DecisionTreeClassifier(max_depth=3, criterion='entropy').fit(X_train, y_train)
model_dtree

Setelah itu, mari kita melakukan prediksi menggunakan data <code>X_test</code>.

In [ ]:
# Prediksi dengan X_test

y_pred = model_dtree.predict(X_test)

Kita dapat menampilkan hasil dari prediksi (<code>y_pred</code>) dan juga hasil sebenarnya (<code>y_test</code>) untuk membandingkan.

In [ ]:
# Menampilkan y_pred dan y_test

print(y_pred [0:5])
print(y_test [0:5])

### Evaluasi

Selanjutnya, kita dapat mengevaluasi kinerja model dengan <code>accuracy_score</code>.

In [ ]:
from sklearn.metrics import accuracy_score

# Menampilkan akurasi
print('Akurasi Decision Tree : ', accuracy_score(y_test, y_pred))

### Tree Visualization

Seperti yang kita ketahui bahwa mode <i>decision tree</i> mengambil keputusan berdasarkan struktur pohon. Nah, kita dapat menampilkan visualisasi pohon tersebut dengan <i>package</i> tambahan yang sudah disediakan oleh Python, yaitu __pydotplus__, dan __python-graphviz__.

Jika Anda belum menginstalnya di komputer Anda, silahkan Anda __HAPUS TANDA PAGAR__ pada blok kode di bawah ini, kemudian jalankan dan tunggu hingga proses selesai.

In [ ]:
#!conda install -c conda-forge pydotplus -y
#!conda install -c conda-forge python-graphviz -y

Sekarang silahkan jalankan 2 blok kode di bawah ini. 

In [ ]:
from six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree

%matplotlib inline 

In [ ]:
dot_data = StringIO()
filename = 'churn_tree.png'
featureNames = df.columns[0:19]
targetNames = df['Churn'].unique().tolist()
out = tree.export_graphviz(model_dtree, feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_train), filled=True,  special_characters=True, rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(200, 100))
plt.imshow(img,interpolation='nearest')

Bagan pohon yang terbentuk akan tersimpan dengan nama __'churn_tree.png'__ di folder yang sama dengan folder notebook ini berada.

<hr>

Copyright @ <a href="https://ilmudatapy.com/">ilmudatapy.com</a>